In [172]:
%%HTML
<style>
  body
  {
    --vscode-font-family: "Courier New";
  }
</style>


<style>
    .header
    {
        background-color: white;
        color: black;
    }
    h1
    {
        text-align: center;
        color : green;
    }
</style>
<div class = "header">
<h1>BÀI TOÁN VỚI CƠ SỞ DỮ LIỆU MOVIELENS 100K</h1>
Cơ sở dư liệu Movielens 100K được công bố năm 1998 bởi <span style = "color:red">GroupLens</span>. Bộ cơ sở dữ liệu này bao gồm 100K đánh giá từ 943 user cho 1682 bộ phim.
Các file quan trọng
- u.data: Chứa toàn bộ đánh giá của 943 người dùng cho 1682 bộ phim. Mỗi người dùng đánh giá ít nhất 20 bộ phim.

- ua.base, ua.test, ub.base, ub.test: là hai cách chia toàn bộ dữ liệu ra thành hai tập con, một tập huấn luyện, một cho kiểm thử. Chúng ta sẽ thực hành trên ua.base và ua.test

- u.user: Chứa thông tin về người dùng bao gồm id, tuổi, giới tính, nghề nghiệp, zipcode (vùng miền), vì những thông tin này cũng có thể ảnh hưởng tới sở thích của các người dùng.

- u.genre: Chứa trên 19 thể loại phim. Các thể loại bao gồm: unknown, Action, Adventure, Animation, Children's, Comedy, Crime, ....

- u.item: thông tin về mỗi bộ phim
</div>


In [130]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [131]:
from __future__ import print_function


In [132]:
# Reading user file
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
user = pd.read_csv('ml-100k/u.user', sep = '|', names = u_cols)

File u.user chứa các thông tin của người dùng bao gồm: age, sex, occupation, zip_code

In [133]:
user

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [134]:
n_user = user.shape[0]
print(n_user)

943


File ua.base và ua.test chứa rating của người dùng cho các bộ phim

In [187]:
# Reading rating file
r_cols = ['user_id', 'moveid', 'rating', 'unix_timestamp']
rating_base = pd.read_csv('./ml-100k/ua.base', sep = '\t', names = r_cols)
rating_test = pd.read_csv('./ml-100k/ua.test', sep = '\t', names = r_cols)
rate_train = rating_base.values
rate_test = rating_test.values


In [210]:
rating_base.head()

,user_id,moveid,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [213]:
rating_base.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90570 entries, 0 to 90569
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   user_id         90570 non-null  int64
 1   moveid          90570 non-null  int64
 2   rating          90570 non-null  int64
 3   unix_timestamp  90570 non-null  int64
dtypes: int64(4)
memory usage: 2.8 MB


In [215]:
rating_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9430 entries, 0 to 9429
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   user_id         9430 non-null   int64
 1   moveid          9430 non-null   int64
 2   rating          9430 non-null   int64
 3   unix_timestamp  9430 non-null   int64
dtypes: int64(4)
memory usage: 294.8 KB


In [137]:
print(rating_base.shape[0])
print(rating_test.shape[0])

90570
9430


File u.item chứa thông tin của các bộ phim

In [177]:
# Reading item files
i_cols = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action',
          'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
          'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols , encoding='latin-1')
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
print('number of items:', items.shape[0])

number of items: 1682


Vì ta đang dựa trên thể loại của phim để xây dựng profile, ta sẽ chỉ quan tâm tới 19 giá trị nhị phân ở mỗi hàng

In [186]:
X0 = items.values
X_train_count = X0[:,-19:]
X_train_count

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

Chúng ta sẽ hiển thị một số hàng đầu tiên của ma trận rate_train

In [188]:
print(rate_train[:4, :])

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]]


Hàng thứ nhất có thể hiểu là user thứ nhất đánh giá moive thứ nhất 5 sao. Cột cuối cùng là thời điểm đánh giá. Chúng ta sẽ bỏ qua thông số này
Tiếp theo chúng ta sẽ xây dựng feature vector cho mỗi item dưạ trên ma trận thể loại phim và feature `TF-IDF` trong thư viện sklearn

In [199]:
from sklearn.feature_extraction.text import TfidfTransformer
transfomer = TfidfTransformer(smooth_idf=True, norm = 'l2')
X = transfomer.fit_transform(X_train_count.tolist()).toarray()


TF-IDF (Term Frequency-Inverse Document Frequency) là một kỹ thuật quan trọng trong xử lý ngôn ngữ tự nhiên và truy xuất thông tin. Ý nghĩa của TF-IDF là giúp đánh giá mức độ quan trọng của một từ trong một tài liệu so với tập hợp các tài liệu khác trong tập dữ liệu
- TF: Tần suất xuất hiện của từ
- IDF: Tần số ngược của văn bản
- Cách tính IDF:
$$IDF(t) = log(\frac{N}{DF(t)})$$

N: Tổng số văn bản trong tập dữ liệu.

DF(t): Tổng số văn bản trong tập dữ liệu chứa từ t.

In [208]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.53676706, 0.65097024, ..., 0.53676706, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

<style>
    .body
    {
        background-color:white;
        color:black;
    }
    h1
    {
        text-align:center;
        color:green;
        font-size:20px;
    }
</style>
<div class = "body">
    <h1>Tìm mô hình cho mỗi user</h1>
    <i>Với mỗi người dùng, chúng ta cần đi tìm những bộ phim nào mà người dùng đó đã đánh giá, và giá trị của các đánh giá đó</i>

</div>

In [204]:
def get_items_rate_by_user(rate_maxtrix, user_id):
    y = rate_maxtrix[:, 0] # all users
    indexs = np.where(y == user_id + 1)[0]
    items_index = rate_maxtrix[indexs, 1] - 1 # movieid start from 0
    score = rate_maxtrix[indexs, 2]
    return items_index, score

In [217]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
d = X.shape[1] # dimension data
W = np.zeros((d, n_user)) # 19 x users
b = np.zeros(n_user) 
for n in range(n_user):
    ids, score = get_items_rate_by_user(rate_train, n)
    model = Ridge(alpha=0.01, fit_intercept=True)
    Xhat = X[ids, :]
    model.fit(Xhat, score)
    W[:, n] = model.coef_
    b[n] = model.intercept_
    

In [220]:
print("W.shape:", W.shape)
print("b.shape:", b.shape)

W.shape: (19, 943)
b.shape: (943,)


In [221]:
Y_hat = X.dot(W) + b

Sau đây chúng ta sẽ lấy ví dụ với 1 người có id = 10

In [231]:
n = 10 # user_id  = 11 
np.set_printoptions(precision = 2)
ids, score = get_items_rate_by_user(rate_test, n)
print('Rated movie ids', ids)
print('True rating', score)
print('Predict rating:',Y_hat[ids, n])

Rated movie ids [ 37 109 110 226 424 557 722 724 731 739]
True rating [3 3 4 3 4 3 5 3 3 4]
Predict rating: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]


In [239]:
rating_test.loc[rating_test['user_id'] == 11]


,user_id,moveid,rating,unix_timestamp
100,11,38,3,891905936
101,11,110,3,891905324
102,11,111,4,891903862
103,11,227,3,891905896
104,11,425,4,891904300
105,11,558,3,891904214
106,11,723,5,891904637
107,11,725,3,891905568
108,11,732,3,891904596
109,11,740,4,891903067


<style>
    .body
    {
        background-color:white;
        color:black;
    }
    h1
    {
        text-align:center;
        color:green;
        font-size:20px;
    }
</style>
<div class = "body">
    <h1>Đánh giá mô hình</h1>
    Để đánh giá mô hình tìm được, chúng ta sẽ sử dụng Root Mean Squared Error (RMSE)
</div>

In [240]:
def envaluate(Yhat, rate, W, b):
    se = cnt = 0
    for n in range(n_user):
        ids, score_truth = get_items_rate_by_user(rate, n)
        score_pred = Y_hat[ids, n]
        e = score_truth - score_pred
        se += (e * e).sum(axis = 0)
        cnt += e.size
    return np.sqrt(se/cnt)
print('RMSE test:', envaluate(Y_hat, rate_test, W, b))
print('RMSE train:', envaluate(Y_hat, rate_train, W, b))


RMSE test: 1.2703282700393037
RMSE train: 0.9089804562826721
